# $k$-means Clustering

In [ ]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import datasets

# import Alison's code for the demo clusters
import src.demo_images as demo
from src.k_means_plotter import k_means

## Learning Goals

- Assess what scenarios could use $k$-means
- Articulate the methodology used by $k$-means
- Apply KMeans from sklearn.cluster to a relevant dataset
- Select the appropriate number of clusters using the elbow method and Silhouette Scores
- Evaluate the weaknesses and remedies to $k$-means

## A Classic Scenario

>You work for the marketing department within a large company that manages a customer base. 
For each customer you have a record of average purchase cost and time since last purchase.<br> 
You know that if you want to retain your customers you cannot treat them the same. You can use targeted marketing ads towards groups that demonstrate different behavior, but how will you divide the customers into groups?

## Clustering!   Finding **GROUPS**

How many groups do you see?

<img src="images/initialscenario.png" width=600>

## Wait - How is clustering different from classification?

>In _classification_ you **know** what groups are in the dataset and the goal is to _**predict**_ class membership accurately.

>In _clustering_ you **do not** know which groups are in the dataset and you are trying to _**identify**_ the groups.

Because we do not have target labels, clustering is a form of machine learning called **unsupervised learning**.

### So what do you do with clustering results?

Clustering is often an *informing* step in your analysis. Once clusters are identified, one can:
- Create strategies on how to approach each group differently
- Use cluster membership as an independent variable in a predictive model
- Use the clusters as the _**target label**_ in future classification models. How would you assign new data to the existing clusters?

## Explore the algorithm with an intuitive K means approach

### Observe the following four methods with a sample dataset:

### Method Questions:

- What do they have in common?
- What are the differences between them?
- How many groups are there in the end?
- Do you see any problems with this method?

| Method 1 | Method 2 |
| -------- | -------- |
| <img src="images/from-left.gif" width=400> | <img src="images/from-right.gif" width=400> |

| Method 3 | Method 4 |
| -------- | -------- |
| <img src="images/from-top.gif" width=400> | <img src="images/from-bottom.gif" width=400> |

In common:

- 


Differences:

- 


Groups:

- 


Problem with this method?

- 


### K-means algorithm, at its core, in an optimization function

<img src="images/minmaxdata.png" width=400>

### Reassigns groups and adjusts centroids to...

<img src="images/min.png" width=700>

### And to...

<img src="images/max.png" width=700>

**Sci-kit Learn** documentation actually has some pretty good [documentation describing the algorithm](https://scikit-learn.org/stable/modules/clustering.html#k-mean) if you wish for more detail.

## The Challenges of Clustering

In [ ]:
X, Y = make_blobs(centers=5, random_state=42)
X[:5, :]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1]);

The value of the `random_state` parameter in our `k_means()` function can make a big difference to the final clusters! We might find what are indeed the intuitive clusters in our data:

In [ ]:
df = k_means(X[:, 0], X[:, 1], k=5, random_state=2)

But we also might get different results:

In [ ]:
df = k_means(X[:, 0], X[:, 1], k=5, random_state=42)

In [ ]:
df = k_means(X[:, 0], X[:, 1], k=5, random_state=3)

And of course we could set $k$ to something other than 5:

In [ ]:
df = k_means(X[:, 0], X[:, 1], k=3)

## **Assumptions** and **challenges** of $k$-means

- Demonstrate the ideal $k$-means dataset
- Show three scenarios where $k$-means struggles

#### Ideal $k$-means scenario

In [ ]:
demo.ideal()

#### Meets all assumptions:

- Independent variables
- Balanced cluster sizes
- Clusters have similar density
- Spherical clusters/equal variance of variables


#### Problem Scenario 1 - classes not all round

In [ ]:
demo.messyOne()

#### Problem Scenario 2 - imbalanced class size

In [ ]:
demo.messyTwo()

#### Problem Scenario 3 - class size and density

In [ ]:
demo.messyThree()

#### Solution to challenges:

- Preprocessing: PCA or scaling
- Try a different clustering methods

### Simple Demonstration

- This is a sample dataset. 
- Let us assume the data is already scaled.

In [ ]:
dummy_dat = pd.read_csv("data/xclara.txt",
                        header=0,
                        index_col=0)

In [ ]:
dummy_dat.head()

#### EDA of variables

In [ ]:
dummy_dat.describe()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(dummy_dat['V1'], dummy_dat['V2']);

#### Introduction of `Kmeans`

In [ ]:
model = KMeans(n_clusters=3).fit(dummy_dat)

In [ ]:
model.cluster_centers_

In [ ]:
fig, ax = plt.subplots()
ax.scatter(dummy_dat['V1'], dummy_dat['V2'])
for i in range(len(model.cluster_centers_)):
    ax.scatter(model.cluster_centers_[i][0],
               model.cluster_centers_[i][1]);

In [ ]:
model.predict([[60, -20]])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(dummy_dat['V1'], dummy_dat['V2'],
           c=model.labels_);

In [ ]:
labeled_df = pd.concat([dummy_dat, 
                        pd.DataFrame(model.labels_, columns=['cluster'])], 
                       axis=1)

In [ ]:
labeled_df.head()

In [ ]:
cluster0 = labeled_df[labeled_df['cluster'] == 0]
cluster1 = labeled_df[labeled_df['cluster'] == 1]
cluster2 = labeled_df[labeled_df['cluster'] == 2]

In [ ]:
cluster0['V1'].head()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(cluster0['V1'], cluster0['V2'], c='k', alpha=0.4)
ax.scatter(cluster1['V1'], cluster1['V2'], c='c', alpha=0.4)
ax.scatter(cluster2['V1'], cluster2['V2'], c='r', alpha=0.4);

## Note
#### You *may* have different cluster centers.

We saw in the demo that the algorithm is sensitive to starting points. It's a good idea to use `random_state` to ensure repeatable results.

## Choosing the appropriate number for $k$

#### Two metrics we can use: *Elbow Method* and the *Silhouette Coefficient*

### Elbow Method

Elbow method uses the sum of squared error (SSE) calculated from each instance of $k$ to find the best value of $k$.

This is sometimes called the "inertia" of the model, and fitted sklearn $k$-means models have an `inertia_` attribute.

Sometimes you will see the SSE divided by the total sum of squares in the dataset (how far is each point from the center of the entire dataset)

Fewer clusters seems better, but _inertia will always decrease with more clusters_. Hence the idea of looking for an _elbow_ in the plot of inertia vs. $k$.

In [ ]:
model.inertia_

Inertia is the sum of squared distances between points and their cluster center.

In [ ]:
# Specifying the dataset and initializing variables
X = dummy_dat
distortions = []

# Calculate inertia for different K
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=301)
    kmeans.fit(X)
    distortions.append(kmeans.inertia_)

# Plot values of inertia
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_title('Elbow curve')
ax.set_xlabel('k')
ax.plot(range(2, 10), distortions)
ax.grid(True)
ax.set_ylabel("Inertia")
ax.set_xlabel("Number of Clusters");

#### How many clusters is best?

- 



### Silhouette Coefficient

![silo](images/silo2.png)

> **a** refers to the average distance between a point and all other points in that cluster.
>
> **b** refers to the average distance between that same point and all other points in clusters to which it does not belong

It is calculated for each point in the dataset, then averaged across all points for one cumulative score.

The Silhouette Coefficient ranges between -1 and 1. The closer to 1, the more clearly defined are the clusters. The closer to -1, the more incorrect assignment.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html



In [ ]:
# Grab the labels
labels = model.labels_

In [ ]:
# Let's score using sklearn's silhouette_score
metrics.silhouette_score(dummy_dat, labels)

In [ ]:
# Finding silhouette scores for many values of k
silhouette_scores = {}

for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=301)
    kmeans.fit(dummy_dat)
    silhouette_scores[k] = metrics.silhouette_score(dummy_dat, kmeans.labels_)

In [ ]:
silhouette_scores

In [ ]:
# Plot Silhouette coefficient
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(range(2, 10), silhouette_scores.values())
ax.axhline(y=np.mean(list(silhouette_scores.values())), color="red", linestyle="--", 
           label="average silhouette score")

ax.set_title('Silhouette coefficients over k')
ax.set_xlabel('k')
ax.set_ylabel('silhouette coefficient')
ax.legend()
ax.grid(True)

In [ ]:
# From a colleague
# This will showcase what each cluster looks like in terms of fit
from matplotlib import cm

km = KMeans(n_clusters=3, random_state=0)
y_km = km.fit_predict(dummy_dat)

cluster_labels = np.unique(y_km)
n_clusters = cluster_labels.shape[0]
silhouette_vals = metrics.silhouette_samples(dummy_dat, y_km, metric='euclidean')
y_ax_lower, y_ax_upper = 0, 0
yticks = []

for i, c in enumerate(cluster_labels):
    c_silhouette_vals = silhouette_vals[y_km == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(float(i) / n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
             edgecolor='none', color=color)

    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(c_silhouette_vals)
    
silhouette_avg = np.mean(silhouette_vals)
plt.axvline(silhouette_avg, color="red", linestyle="--") 

plt.yticks(yticks, cluster_labels + 1)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')

plt.tight_layout()
plt.show()

### Clustering Review:

No right answers! Just looking for patterns.

Evaluate goodness-of-fit using either elbow scores or silhouette scores.

Want elbow score to be low (but not minimum), silhouette score to be high.

#### Useful Resource!

The Yellowbrick library has some of these plots already built out nicely: https://www.scikit-yb.org/en/latest/api/cluster/index.html

## Exercise:

Using online retail data data from [UCI database](https://archive.ics.uci.edu/ml/datasets/online+retail).

You are looking for patterns so you can get people to buy more, more frequently. 

You'll have to do some cleaning, and you might want to create some new variables.

In [ ]:
shopping = pd.read_csv('data/OnlineRetail.csv')

In [ ]:
shopping.head()

In [ ]:
shopping.info()

### Review $k$-means steps
1. Look at and clean data (if needed)
2. Scale data
3. Try various values of $k$
4. Plot SSE and Silhouette coefficient to find best $k$
5. Describe the characteristics of each cluster using their centroids

In [ ]:
# Look at and clean data


In [ ]:
# Scale


In [ ]:
# Try values for k


In [ ]:
# Find best k


In [ ]:
# Explore centroids


### How many clusters fit the data?

What can you tell me about them?

- 
